# AGE-it - Exploratory Data Analysis

- Summary of the recordings - how many hours/days for each sensor?

### Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import glob

%matplotlib inline
#%matplotlib qt
mpl.rcParams['lines.linewidth'] = 0.91
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib qt

from avro.datafile import DataFileReader
from avro.io import DatumReader

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool, Slider, Select
from bokeh.layouts import gridplot
from bokeh.models import Range1d
from bokeh.io import export_png
from bokeh.models import DatetimeTickFormatter

## GENEActiv
Files are already converted into .bin with the bin2parquet notebook

In [2]:
data_path = "/Users/augenpro/Documents/Age-IT/data/" # path to the folder containing the subjects
participants = sorted([p for p in os.listdir(data_path) if not p.startswith(".")]) # list of the participants
visit = "T0 (baseline)" # T0 (baseline), T1 (follow-up @ 6 months), T2 (follow-up @ 12 months)

# participants = participants[5:6] # select the first participant

sensors = ["GeneActivPolso", "GeneActivCaviglia"]#, "RootiRx"]

geneactiv = {}

for participant in participants:
    print(participant)
    for sensor in sensors:
        # print(sensor)
        # Load the data
        path = os.path.join(data_path, participant, visit, sensor)
        files = os.listdir(os.path.join(data_path, participant, visit, sensor))
        # if there are already parquet files, skip
        for f in files:
            if f.endswith(".parquet"):
                # print(f)
                acc_df = pd.read_parquet(os.path.join(path, f))
                # geneactiv[sensor] = acc_df
                if len(acc_df) == 0:
                    print(f"{sensor}: is empty")
                else:
                    print(f"{sensor}:  {acc_df.index[-1] - acc_df.index[0]}".split(".")[0])

    print("")
# # Plot the data
# p = figure(plot_width=800, plot_height=400, x_axis_type="datetime", title=f"{participant} - {sensor} - {visit}")
# p.line(data["timestamp"], data["x"], line_width=2, legend_label="x", color="blue")
# p.line(data["timestamp"], data["y"], line_width=2, legend_label="y", color="green")
# p.line(data["timestamp"], data["z"], line_width=2, legend_label="z", color="red")
# p.xaxis.formatter = DatetimeTickFormatter(days="%d/%m %H:%M", hours="%H:%M")
# p.legend.location = "top_left"
# p.legend.click_policy="hide"
# show(p)
# export_png(p, filename=f"{participant}_{sensor}_{visit}.png")
# print(f"{participant}_{sensor}_{visit}.png")
# print("")

08623
GeneActivPolso:  7 days 01:20:20
GeneActivCaviglia:  7 days 01:19:44

08667

14219
GeneActivPolso:  6 days 22:27:29
GeneActivCaviglia:  6 days 22:27:26

20603

23483
GeneActivPolso:  6 days 23:38:14
GeneActivCaviglia:  6 days 23:38:20

36644
GeneActivPolso:  6 days 22:46:32
GeneActivCaviglia:  6 days 22:47:14

36765

36920
GeneActivPolso:  5 days 23:55:05
GeneActivCaviglia:  5 days 23:54:05

58319
GeneActivPolso:  0 days 18:23:38
GeneActivCaviglia:  7 days 01:19:05

59794


: 

In [ ]:
# Recording duration
for sensor, data in geneactiv.items():
    print(f"{sensor}: {data.index[-1] - data.index[0]}".split(".")[0])

GeneActivPolso: 6 days 22:46:32
GeneActivCaviglia: 6 days 22:47:14


In [ ]:
# geneactiv["GeneActivPolso"]["acc_SMV"] = np.sqrt(geneactiv["GeneActivPolso"]["x"]**2 + geneactiv["GeneActivPolso"]["y"]**2 + geneactiv["GeneActivPolso"]["z"]**2)
# geneactiv["GeneActivCaviglia"]["acc_SMV"] = np.sqrt(geneactiv["GeneActivCaviglia"]["x"]**2 + geneactiv["GeneActivCaviglia"]["y"]**2 + geneactiv["GeneActivCaviglia"]["z"]**2)

plt.figure(figsize = (19,11))
plt.subplot(2, 1, 1)
plt.plot(geneactiv["GeneActivPolso"]["acc_SMV"].iloc[:1000000], label = "GeneActivPolso")
plt.plot(geneactiv["GeneActivCaviglia"]["acc_SMV"].iloc[:1000000], label = "GeneActivCaviglia")
plt.title("Recording Start", fontsize = 21)
plt.legend()
plt.subplot(2, 1, 2)
plt.plot(geneactiv["GeneActivPolso"]["acc_SMV"].iloc[-1000000:], label = "GeneActivPolso")
plt.plot(geneactiv["GeneActivCaviglia"]["acc_SMV"].iloc[-1000000:], label = "GeneActivCaviglia")
plt.title("Recording End", fontsize = 21)
plt.legend()

## Verity Sense

In [ ]:
from data_io.veritysense.convert_polar import process_polar

In [ ]:
import fastavro

def read_avro_veritysense(file_path, offset_vs = 946684800000000000+ 3600 * 1e9):
    """
    Reads an Avro file and returns the data as a pandas DataFrame.
    """
    with open(file_path, "rb") as avro_file:
        # Use fastavro.reader to read the Avro file
        reader = fastavro.reader(avro_file)
        # Convert the records to a list
        records = [record for record in reader]
       # to dataframe
        df = pd.DataFrame(records)
        df.index = pd.to_datetime(df["timestamp"] + offset_vs, unit="ns") 
        df.drop("timestamp", axis=1, inplace=True)
    return df / 1000

In [ ]:
import re
from datetime import datetime

# Function to extract full datetime (including time) for sorting
def extract_datetime(filename):
    match = re.search(r'(\w{3}) (\w{3}) (\d{2}) (\d{2})-(\d{2})-(\d{2}) .* (\d{4})', filename)
    if match:
        weekday, month, day, hour, minute, second, year = match.groups()
        date_str = f"{day} {month} {year} {hour}:{minute}:{second}"
        date_obj = datetime.strptime(date_str, "%d %b %Y %H:%M:%S")  # Convert to datetime object
        return date_obj
    return datetime.max  # Default to a max value if parsing fails

In [ ]:
data_path = "/Users/augenpro/Documents/Age-IT/data" # path to the folder containing the subjects
participants = sorted([p for p in os.listdir(data_path) if not p.startswith(".")]) # list of the participants
visit = "T0 (baseline)" # T0 (baseline), T1 (follow-up @ 6 months), T2 (follow-up @ 12 months)

# participants = participants[0:1] # select the first participant

participants = ["23483"]

sensor = "VeritySense"#, "GeneActivPolso", "GeneActivCaviglia", "RootiRx"]

for participant in participants:
    acc_df = pd.DataFrame()
    ppg_df = pd.DataFrame()
    # print(sensor)
    path = os.path.join(data_path, participant, visit, sensor)
    files_in_path = [f for f in os.listdir(path) if not f.startswith(".")]
    if len(files_in_path) <= 2: # "_" and "AVRO"
        continue

    print(f"**************** {participant} ****************")
    acc_path = os.path.join(path, "AVRO/acc")
    ppg_path = os.path.join(path, "AVRO/ppg")
    for f in sorted(os.listdir(acc_path), key=extract_datetime):
        current_acc = read_avro_veritysense(os.path.join(acc_path, f))
        acc_df = pd.concat([acc_df, current_acc])

    for f in sorted(os.listdir(ppg_path), key=extract_datetime):
        current_ppg = read_avro_veritysense(os.path.join(ppg_path, f))
        ppg_df = pd.concat([ppg_df, current_ppg])

    t_acc_rec = acc_df.index[acc_df.index.to_series().diff().dt.total_seconds() > 0.5]
    t_acc_disc = acc_df.index[np.where(acc_df.index.to_series().diff().dt.total_seconds() > 0.5)[0]-1]
    t_disc_rec_acc = pd.DataFrame({"disc": t_acc_disc, "rec": t_acc_rec})
    total_duration_acc = acc_df.index[-1] - acc_df.index[0]
    disconnetions_duration_acc = t_disc_rec_acc["rec"] - t_disc_rec_acc["disc"]
    effective_duration_acc = total_duration_acc - disconnetions_duration_acc.sum()
    if len(t_acc_disc) > 0:
        print("\n ***** Disconnections in ACC *****")
        # for i in range(len(t_acc_disc)):
        #     print(f"Disconnection {i+1}:")
        #     print(f"Start: {t_acc_disc[i].strftime('%Y-%m-%d %H:%M:%S')}")
        #     print(f"End: {t_acc_rec[i].strftime('%Y-%m-%d %H:%M:%S')}")
        #     print(f"Duration: {str(t_acc_rec[i] - t_acc_disc[i]).split(' ')[2][3:5]} min and {str(t_acc_rec[i] - t_acc_disc[i]).split(' ')[2][6:8]} s")
        print("Number of disconnections: " + str(len(t_acc_disc)))
        print("Total duration of disconnections: " + str(np.sum([t_acc_rec[i] - t_acc_disc[i] for i in range(len(t_acc_disc))])).split(' ')[2][:8] + " (hours, minutes and seconds)")
        print(f"ACC effective duration: {effective_duration_acc}".split(".")[0])
    # Fill disconnection portions with NaNs and linearly interpolate
    for start, end in zip(t_acc_disc, t_acc_rec):
        acc_df.loc[start:end] = np.nan
    acc_df = acc_df.interpolate(method='time')

    t_ppg_rec = ppg_df.index[ppg_df.index.to_series().diff().dt.total_seconds() > 1]
    t_ppg_disc = ppg_df.index[np.where(ppg_df.index.to_series().diff().dt.total_seconds() > 1)[0]-1]
    t_disc_rec_ppg = pd.DataFrame({"disc": t_ppg_disc, "rec": t_ppg_rec})
    total_duration_ppg = ppg_df.index[-1] - ppg_df.index[0]
    disconnetions_duration_ppg = t_disc_rec_ppg["rec"] - t_disc_rec_ppg["disc"]
    effective_duration_ppg = total_duration_ppg - disconnetions_duration_ppg.sum()
    if len(t_ppg_disc) > 0:
        print("\n ***** Disconnections in PPG *****")
        # for i in range(len(t_ppg_disc)):
        #     print(f"Disconnection {i+1}:")
        #     print(f"Start: {t_ppg_disc[i].strftime('%Y-%m-%d %H:%M:%S')}")
        #     print(f"End: {t_ppg_rec[i].strftime('%Y-%m-%d %H:%M:%S')}")
        #     print(f"Duration: {str(t_ppg_rec[i] - t_ppg_disc[i]).split(' ')[2][3:5]} min and {str(t_ppg_rec[i] - t_ppg_disc[i]).split(' ')[2][6:8]} s")
        print("Number of disconnections: " + str(len(t_ppg_disc)))
        print("Total duration of disconnections: " + str(np.sum([t_ppg_rec[i] - t_ppg_disc[i] for i in range(len(t_ppg_disc))])).split(' ')[2][:8] + " (hours, minutes and seconds)")
        print(f"PPG effective duration: {effective_duration_ppg}".split(".")[0])
        print("")
    # Fill disconnection portions with NaNs and linearly interpolate
    for start, end in zip(t_ppg_disc, t_ppg_rec):
        ppg_df.loc[start:end] = np.nan
    ppg_df = ppg_df.interpolate(method='time')

**************** 23483 ****************

 ***** Disconnections in ACC *****
Number of disconnections: 134
Total duration of disconnections: 06:41:26 (hours, minutes and seconds)
ACC effective duration: 4 days 16:50:17

 ***** Disconnections in PPG *****
Number of disconnections: 137
Total duration of disconnections: 06:11:44 (hours, minutes and seconds)
PPG effective duration: 4 days 17:19:59



##### Soggetto 97060 ------- weird timestamps starting at 1970

In [ ]:
total_dur = acc_df.index[-1] - acc_df.index[0]
disconnetions_dur = t_disc_rec_acc["rec"] - t_disc_rec_acc["disc"]
effective_dur = total_dur - disconnetions_dur.sum()
total_dur, disconnetions_dur.sum(), effective_dur

(Timedelta('6 days 23:31:43.876258560'),
 Timedelta('2 days 06:41:26.693904128'),
 Timedelta('4 days 16:50:17.182354432'))

In [ ]:
for start, end in zip(t_acc_disc, t_acc_rec):
    print(start,end)
    break

ppg_df.loc[start:end]

2025-01-21 16:17:29.975070208 2025-01-21 16:20:03.052938240


,ppg1,ppg2,ppg3,ambient
timestamp,,,,


In [ ]:
ppg_df.loc[pd.Timestamp("2025-01-21 16:20:03.052938240"):pd.Timestamp("2025-01-22 16:20:03.052938240")].to_csv("ppg_df.csv")

In [ ]:
ppg = pd.read_pickle("ppg_df.pkl")
ppg

,ppg1,ppg2,ppg3,ambient
timestamp,,,,
2025-01-21 16:20:03.344035328,115.751091,19.832478,75.259832,-291.321995
2025-01-21 16:20:03.362216960,115.752000,19.834000,75.260000,-291.322000
2025-01-21 16:20:03.380399104,115.551000,19.657000,75.209000,-291.231000
2025-01-21 16:20:03.398580736,115.589000,19.481000,75.181000,-291.230000
2025-01-21 16:20:03.416762624,115.991000,19.511000,75.189000,-291.263000
...,...,...,...,...
2025-01-22 16:20:02.974105088,63.964000,22.219000,25.433000,-291.274000
2025-01-22 16:20:02.992358400,65.248000,23.506000,26.564000,-291.263000
2025-01-22 16:20:03.010611712,66.445000,24.733000,27.541000,-291.279000


In [ ]:
import neurokit2 as nk
ppg_df["ppg_filt"] = -nk.signal_filter(ppg_df["ppg1"], lowcut=0.5, highcut=8, sampling_rate=55)

In [ ]:
plt.figure(figsize = (19,11))
plt.subplot(2, 1, 1)
plt.plot(acc_df)
plt.title("Acceleration")
# plt.legend(acc_df.columns)
plt.subplot(2, 1, 2, sharex = plt.subplot(2, 1, 1))
plt.plot(ppg_df["ppg1"])
plt.title("PPG")
# plt.legend(["ppg_filt"])

Text(0.5, 1.0, 'PPG')

In [ ]:
# Calculate the actual recording duration by summing up the differences between consecutive timestamps
time_diffs = acc_df.index.to_series().diff().dropna()
overall_duration = time_diffs.sum()
print(f"Overall duration of the recording (excluding gaps): {overall_duration}")

Overall duration of the recording (excluding gaps): 0 days 05:29:49.749643520


In [ ]:
ppg_df.index[-1] - ppg_df.index[0]

Timedelta('0 days 05:29:53.340454144')

## Rooti

In [ ]:
data_path = "/Users/augenpro/Documents/Age-IT/data/" # path to the folder containing the subjects
participants = sorted([p for p in os.listdir(data_path) if not p.startswith(".")]) # list of the participants
visit = "T0 (baseline)" # T0 (baseline), T1 (follow-up @ 6 months), T2 (follow-up @ 12 months)
participants = ["14219"]
# participants = participants[5:6] # select the first participant

sensors = ["RootiRx"]#, "RootiRx"]

sensor = sensors[0]

rootirx = {}

for participant in participants:
    path = os.path.join(data_path, participant, visit, sensor)
    files = os.listdir(os.path.join(data_path, participant, visit, sensor))

    files = [f for f in files if f == participant]

    if len(files) == 0:
        # print("No RootiRx data")
        continue
    
    print("***** " + participant + " *****")
    rooti_path = os.path.join(data_path, participant, visit, sensor, files[0], "measure")

    raw_acc_path = [x[0] for x in os.walk(rooti_path) if x[0].endswith("/GSENSOR")][0]

    # print(raw_acc_path)

    acc_files = sorted(os.listdir(raw_acc_path))

    # print(len(acc_files))

    acc = pd.DataFrame()

    acc_first = pd.read_csv(raw_acc_path + '/' + acc_files[0], compression='zip', index_col = 0, header = None, names = ["packet", "acc_x", "acc_y", "acc_z", "idk", "idk2"])
    acc_first_start = pd.to_datetime(acc_first.index[0], unit = "s")

    acc_last = pd.read_csv(raw_acc_path + '/' + acc_files[-1], compression='zip', index_col = 0, header = None, names = ["packet", "acc_x", "acc_y", "acc_z", "idk", "idk2"])
    acc_last_end = pd.to_datetime(acc_last.index[-1], unit = "s")

    print(f"Total duration: {acc_last_end - acc_first_start}")

    for i, acc_file in enumerate(acc_files):
        acc1 = pd.read_csv(raw_acc_path + '/' + acc_file, compression='zip', index_col = 0, header = None, names = ["packet", "acc_x", "acc_y", "acc_z", "idk", "idk2"])
        acc1 = acc1[["acc_x", "acc_y", "acc_z"]]
        acc1.index = pd.date_range(start = pd.to_datetime(acc1.index[0], unit = "s"), periods = len(acc1), freq = "0.032s")
        acc = pd.concat([acc, acc1])

    # print(path)

    print("")
    break

    # print(files)

***** 14219 *****
Total duration: 6 days 23:49:20



In [20]:
plt.figure(figsize = (19,11))
plt.plot(np.diff(acc.index).astype('timedelta64[s]'))

In [ ]:
acc1.head()

,packet,acc_x,acc_y,acc_z,idk,idk2
2025-01-17 10:34:21.000000,0,176,-191,1288,18,166
2025-01-17 10:34:21.003200,1,150,-183,1324,18,166
2025-01-17 10:34:21.006400,2,150,119,1187,18,166
2025-01-17 10:34:21.009600,3,115,-164,1379,18,166
2025-01-17 10:34:21.012800,4,60,-287,1468,18,166


# Synch between GENEActiv and Rooti

In [6]:
data_path = "/Users/augenpro/Documents/Age-IT/data/" # path to the folder containing the subjects
participants = sorted([p for p in os.listdir(data_path) if not p.startswith(".")]) # list of the participants
visit = "T0 (baseline)" # T0 (baseline), T1 (follow-up @ 6 months), T2 (follow-up @ 12 months)

# participants = participants[5:6] # select the first participant

sensors = ["GeneActivPolso", "GeneActivCaviglia"]#, "RootiRx"]

geneactiv = {}

participants = ["14219"]

for participant in participants:
    print(participant)
    for sensor in sensors:
        path = os.path.join(data_path, participant, visit, sensor)
        files = os.listdir(os.path.join(data_path, participant, visit, sensor))
        # if there are already parquet files, skip
        for f in files:
            if f.endswith(".parquet"):
                # print(f)
                acc_gen = pd.read_parquet(os.path.join(path, f))
                # geneactiv[sensor] = acc_gen
                if len(acc_gen) == 0:
                    print(f"{sensor}: is empty")
                else:
                    print(f"{sensor}:  {acc_gen.index[-1] - acc_gen.index[0]}".split(".")[0])
                if sensor == "GeneActivPolso":
                    geneactiv["wrist"] = acc_gen
                elif sensor == "GeneActivCaviglia":
                    geneactiv["ankle"] = acc_gen
                del acc_gen
    print("")

# Rooti
sensor = "RootiRx"
for participant in participants:
    path = os.path.join(data_path, participant, visit, sensor)
    files = os.listdir(os.path.join(data_path, participant, visit, sensor))

    files = [f for f in files if f == participant]

    if len(files) == 0:
        # print("No RootiRx data")
        continue
    
    print("***** " + participant + " *****")
    rooti_path = os.path.join(data_path, participant, visit, sensor, files[0], "measure")

    raw_acc_path = [x[0] for x in os.walk(rooti_path) if x[0].endswith("/GSENSOR")][0]
    acc_files = sorted(os.listdir(raw_acc_path))
    acc_rooti = pd.DataFrame()
    t_acc_rooti = pd.Series()
    first_acc_file = pd.read_csv(raw_acc_path + '/' + acc_files[0], compression='zip', index_col = 0, header = None)
    t_start_rooti = pd.to_datetime(first_acc_file.index[0], unit = "s")
    for i, acc_file in enumerate(acc_files):
        acc1 = pd.read_csv(raw_acc_path + '/' + acc_files[i], compression='zip', index_col = 0, header = None) / 1000 # convert to g
        t_acc_start = pd.to_datetime(acc1.index[0], unit = "s")
        t_acc_end = pd.to_datetime(acc1.index[-1]+1, unit = "s")
        acc1 = acc1.iloc[:, [1, 2, 3]].reset_index(drop = True)
        acc1.columns = ["acc_x", "acc_y", "acc_z"]
        t_acc1 = pd.date_range(start = t_acc_start, end = t_acc_end, periods=len(acc1)+1)[:-1]
        acc_rooti = pd.concat([acc_rooti, acc1])
        t_acc_rooti = pd.concat([t_acc_rooti, t_acc1.to_series()])

    ecg_filtered_path = [x[0] for x in os.walk(rooti_path) if x[0].endswith("/FilteredECG")][0] + "/250"
    ecg_filtered_files = sorted(os.listdir(ecg_filtered_path))
    ecg_dataframes = []

    # Iterate through ecg filtered files
    for i, ecg_file in enumerate(ecg_filtered_files):
        ecg_filtered_df = pd.read_csv(
            ecg_filtered_path + '/' + ecg_file, 
            compression='zip', 
            header=None, 
            names=["ecg", "Rpeaks"]
        )
        # t_start = pd.to_datetime(ecg_filtered_files[0].split(".")[0], unit = "s")
        # ecg_filtered_df.index = pd.date_range(start = t_start, periods = len(ecg_filtered_df), freq = "0.004s")
        ecg_dataframes.append(ecg_filtered_df)
    t_start_ecg = pd.to_datetime(int(ecg_filtered_files[0].split(".")[0]), unit = "s")
    ecg_filtered = pd.concat(ecg_dataframes, axis=0)
    ecg_filtered.index = pd.date_range(start = t_start_ecg, periods = len(ecg_filtered), freq = "0.004s")

    # print(path)

    print("")

14219
GeneActivPolso:  6 days 22:27:29
GeneActivCaviglia:  6 days 22:27:26

***** 14219 *****


/var/folders/8x/dvs9gy2x1fq3xdz6r9lt_t_40000gp/T/ipykernel_42501/1118766901.py:64: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  t_acc_rooti = pd.concat([t_acc_rooti, t_acc1.to_series()])


In [11]:
ecg_filtered.to_parquet(ecg_filtered_path + "/ecg_filtered.parquet")

In [ ]:
# plt.figure()
# plt.plot(t_acc_rooti.diff().dropna().dt.total_seconds())

plt.figure()
plt.plot(ecg_filtered.index.diff().dropna().total_seconds())

In [4]:
ecg_filtered.head()

,ecg,Rpeaks
2025-01-17 10:34:21.000,16.445,NaN
2025-01-17 10:34:21.004,5.801,NaN
2025-01-17 10:34:21.008,-1.034,NaN
2025-01-17 10:34:21.012,-2.867,NaN
2025-01-17 10:34:21.016,-1.026,NaN


In [5]:
ecg_filtered["ecg"].index[0], t_acc_rooti.iloc[0]

(Timestamp('2025-01-24 10:14:21'), Timestamp('2025-01-17 10:34:21'))

In [10]:
plt.figure(figsize = (19,11))
plt.subplot(2, 1, 1)
plt.plot(ecg_filtered["ecg"].loc[pd.Timestamp("2025-01-17 11:34:21"):pd.Timestamp("2025-01-18 11:34:21") + pd.Timedelta(hours = 1)])
plt.subplot(2, 1, 2, sharex = plt.subplot(2, 1, 1))
plt.plot(acc_rooti.loc[pd.Timestamp("2025-01-17 11:34:21"):pd.Timestamp("2025-01-18 11:34:21") + pd.Timedelta(hours = 1)])

In [8]:
acc_rooti.index = t_acc_rooti.index #+ pd.Timedelta("1h")

In [38]:
from utils.compute_acc_SMV import compute_acc_SMV

acc_rooti["acc_SMV"] = compute_acc_SMV(acc_rooti)
acc_gen["acc_SMV"] = compute_acc_SMV(acc_gen)

In [39]:
import seaborn as sns
sns.set_context("talk")

In [48]:
plt.figure(figsize = (19,11))
plt.plot(acc_gen["acc_SMV"].iloc[:500000], label = "GENEActiv", linewidth = 0.91)
plt.plot(acc_rooti["acc_SMV"].iloc[:500000], label = "RootiRx", linewidth = 0.91)
# plt.legend()

## Script that check for cross-correlations between different sensors, and adjust the start time accordingly

Assumption: the rotations are done within the first hour of recording

1. Resample the first hour of the signals to the same number of samples (for geneactiv 100 Hz)
2. Segment signal in 20 second windows with an overlap of 10s (the rotations last 10 seconds)
3. Compute the CCF for each of the windows 
4. Find the maximum and plot?

In [22]:
# 1. Resample the overall Rooti signal to the same number of samples (for geneactiv 100 Hz)

from utils.resample_signal import apply_resample

fs_res = 100

t_acc_rooti_resampled, acc_rooti_resampled = apply_resample(time = acc_rooti.index.astype(np.int64).to_numpy(), time_rs = acc_gen.index.astype(np.int64).to_numpy(), 
                                     data = acc_rooti["acc_SMV"].values)

acc_rooti_resampled = pd.Series(acc_rooti_resampled[0], index = pd.to_datetime(t_acc_rooti_resampled))

In [41]:
# 1. Resample the first hour of the signals to the same number of samples (for geneactiv 100 Hz)

from utils.resample_signal import apply_resample

fs_res = 100

acc_rooti_first_hour = acc_rooti.loc[:t_start_rooti + pd.Timedelta("2h")]["acc_SMV"]
acc_gen_first_hour = acc_gen.loc[:acc_gen.index[0] + pd.Timedelta("1h")]["acc_SMV"]

t_acc_rooti_resampled, acc_rooti_resampled = apply_resample(time = acc_rooti_first_hour.index.astype(np.int64).to_numpy(), time_rs = acc_gen_first_hour.index.astype(np.int64).to_numpy(), 
                                     data = acc_rooti_first_hour.values)

acc_rooti_resampled = pd.Series(acc_rooti_resampled[0], index = pd.to_datetime(t_acc_rooti_resampled))

In [ ]:
# 2. Segment the signals into 20s windows with 50% overlap

from utils.segment_signal import segment_signal

acc_gen_segments = segment_signal(acc_gen_first_hour, window_size = 20*fs_res, overlap = 0.5)
# acc_rooti_segments = segment_signal(acc_rooti_resampled, window_size = 20*fs_res, overlap = 0.5)

In [43]:
# 3. Compute the crosscorrelation between the two signals for each window
from utils.crosscorr import crosscorr

offsets = []
max_corr = []
lags = np.arange(-400, 401, 1) # 400 samples = 4s, resolution of 1 samples = 0.01s
for i, (gen, rooti) in enumerate(zip(acc_gen_segments, acc_rooti_segments)):
    if len(gen) != len(rooti):
        continue
    ccf = [crosscorr(gen, rooti, lag) for lag in lags]
    max_corr.append(np.max(ccf))
    offset = np.argmax(ccf) - lags[-1]
    offset_s = offset / fs_res
    offsets.append(offset_s)

In [44]:
plt.figure(figsize = (11,5))
plt.subplot(2, 1, 1)
plt.plot(max_corr, '-*', label = "Max correlation")
plt.xlabel("Window")
plt.ylabel("Correlation")
plt.legend()
plt.subplot(2, 1, 2, sharex = plt.subplot(2, 1, 1))
plt.plot(offsets, '-*', label = "Offset (s)")
plt.xlabel("Window")
plt.ylabel("Offset (s)")
plt.legend()

In [46]:
# Find the maximum correlation and the corresponding offset
max_corr = np.array(max_corr)
offsets = np.array(offsets)
max_corr_idx = np.argmax(max_corr)
max_corr_val = max_corr[max_corr_idx]

# Apply the offset to the RootiRx signal
offset = offsets[max_corr_idx]

acc_rooti_synched = acc_rooti.copy()
acc_rooti_synched.index = acc_rooti_synched.index - pd.Timedelta(f"{offset}s")
# Plot the signals
plt.figure(figsize = (19,11))
plt.plot(acc_gen["acc_SMV"].iloc[:500000], label = "GENEActiv ACC SMV")
plt.plot(acc_rooti_synched["acc_SMV"].iloc[:500000], label = "RootiRx ACC SMV")
plt.ylabel("Acceleration (g)")
plt.legend(loc = "upper left")

## Do it for all GENEActiv

In [12]:
geneactiv.keys()

dict_keys(['wrist', 'ankle'])

In [14]:
from utils.compute_acc_SMV import compute_acc_SMV

# Compute SMV for the GENEActiv signals
geneactiv["wrist"]["acc_SMV"] = compute_acc_SMV(geneactiv["wrist"])
geneactiv["ankle"]["acc_SMV"] = compute_acc_SMV(geneactiv["ankle"])

In [32]:
acc_gen_wirst_first_hour = geneactiv["wrist"].loc[:geneactiv["wrist"].index[0] + pd.Timedelta("5min")]["acc_SMV"]
acc_gen_ankle_first_hour = geneactiv["ankle"].loc[:geneactiv["ankle"].index[0] + pd.Timedelta("5min")]["acc_SMV"]

# 1. Resample the first hour of the signals to the same number of samples (for geneactiv 100 Hz)
fs_res = 100

from utils.resample_signal import apply_resample

t_acc_gen_ankle_resampled, acc_gen_ankle_resampled = apply_resample(time = acc_gen_ankle_first_hour.index.astype(np.int64).to_numpy(), time_rs = acc_gen_wirst_first_hour.index.astype(np.int64).to_numpy(), 
                                     data = acc_gen_ankle_first_hour.values)

acc_gen_ankle_resampled = pd.Series(acc_gen_ankle_resampled[0], index = pd.to_datetime(t_acc_gen_ankle_resampled))

# 2. Segment the signals into 20s windows with 50% overlap

from utils.segment_signal import segment_signal

acc_gen_wrist_segments = segment_signal(acc_gen_wirst_first_hour, window_size = 20*fs_res, overlap = 0.5)
acc_gen_ankle_segments = segment_signal(acc_gen_ankle_resampled, window_size = 20*fs_res, overlap = 0.5)

In [33]:
acc_gen_wirst_first_hour

2025-01-17 11:38:44.000000000    0.987872
2025-01-17 11:38:44.009999990    1.025177
2025-01-17 11:38:44.019999981    1.045466
2025-01-17 11:38:44.029999971    1.076971
2025-01-17 11:38:44.039999962    1.043428
                                   ...   
2025-01-17 11:43:43.960000038    1.005571
2025-01-17 11:43:43.970000029    1.005720
2025-01-17 11:43:43.980000019    1.012752
2025-01-17 11:43:43.990000010    1.011449
2025-01-17 11:43:44.000000000    1.015252
Name: acc_SMV, Length: 30001, dtype: float64

In [35]:
# 3. Compute the crosscorrelation between the two signals for each window

from utils.crosscorr import crosscorr

offsets = []
max_corr = []
lags = np.arange(-400, 401, 1) # 400 samples = 4s, resolution of 1 samples = 0.01s
for i, (wrist, ankle) in enumerate(zip(acc_gen_wrist_segments, acc_gen_ankle_segments)):
    if len(wrist) != len(ankle):
        continue
    ccf = [crosscorr(wrist, ankle, lag) for lag in lags]
    max_corr.append(np.max(ccf))
    offset = np.argmax(ccf) - lags[-1]
    offset_s = offset / fs_res
    offsets.append(offset_s)

plt.figure(figsize = (11,5))
plt.subplot(2, 1, 1)
plt.plot(max_corr, '-*', label = "Max correlation")
plt.xlabel("Window")
plt.ylabel("Correlation")
plt.legend()
plt.subplot(2, 1, 2, sharex = plt.subplot(2, 1, 1))
plt.plot(offsets, '-*', label = "Offset (s)")
plt.xlabel("Window")
plt.ylabel("Offset (s)")
plt.legend()

In [ ]:
# Find the maximum correlation and the corresponding offset
max_corr = np.array(max_corr)
offsets = np.array(offsets)
max_corr_idx = np.argmax(max_corr)
max_corr_val = max_corr[max_corr_idx]
offset = offsets[max_corr_idx]

# Apply the offset to the GENEActiv ankle signal
acc_gen_ankle_synched = geneactiv["ankle"].copy()
acc_gen_ankle_synched.index = acc_gen_ankle_synched.index - pd.Timedelta(f"{offset}s")

# Plot the signals
plt.figure(figsize = (19,11))
plt.plot(geneactiv["wrist"]["acc_SMV"].iloc[:500000], label = "GENEActiv wrist ACC SMV")
plt.plot(acc_gen_ankle_synched["acc_SMV"].iloc[:500000], label = "GENEActiv ankle ACC SMV")
plt.ylabel("Acceleration (g)")
plt.legend(loc = "upper left")

In [37]:
geneactiv["wrist"].head()

,x,y,z,acc_SMV
2025-01-17 11:38:44.000000000,-0.143220,-0.975911,0.054555,0.987872
2025-01-17 11:38:44.009999990,-0.131305,-1.015662,0.046677,1.025177
2025-01-17 11:38:44.019999981,-0.131305,-1.035537,0.058495,1.045466
2025-01-17 11:38:44.029999971,-0.127333,-1.063362,0.113641,1.076971
2025-01-17 11:38:44.039999962,-0.119390,-1.031562,0.101824,1.043428


In [ ]:
from nonwear.DETACH import nimbaldetach

# Apply the non-wear detection algorithm to the GENEActiv wrist signal
start_stop_nw, _ = nimbaldetach(acc['x'].values, acc['y'].values, acc['z'].values, temp["temp"].values, accel_freq=64, temperature_freq=1, quiet=True)